In [2]:
import requests
import KOBO_URL
import time
import re
import pandas as pd
from bs4 import BeautifulSoup

# スクレイピング対象のサイトのURLを取得
url1 = KOBO_URL.KOBO_URL1
res = requests.get(url1)

# WebサイトのHTML要素を取得
soup = BeautifulSoup(res.text, 'html.parser')

# li要素を取得
soup_li = soup.find_all('li', attrs={'class': 'contest-list-item'})

# 応募リンクを配列に格納する
link_list = []
while_cnt = 0
while True:
    try:
        soup_a = soup_li[while_cnt].find('a')
        soup_a = soup_a.get('href')
        link_list.append(soup_a)
        
    except IndexError as e:
        break
        
    while_cnt += 1


kobo_title_list = []
kobo_feature_list = []
kobo_info_list = []


for n in range(len(link_list)):
# for n in range(2):
    # 情報を取得する公募ページに移動してHTTPを取得
    kobo_url = link_list[n]
    res_kobo = requests.get(kobo_url)
    soup_kobo = BeautifulSoup(res_kobo.text, 'html.parser')
    soup_kobo_dl = soup_kobo.find_all('dl')
    
    while_cnt = 0
    while True:
        try:
            soup_kobo_dt = soup_kobo_dl[while_cnt].find_all('dt')
            if '締切' == soup_kobo_dt[0].text:
                break
        except IndexError as e:
            break
        while_cnt += 1
    
    soup_kobo_dd = soup_kobo_dl[while_cnt].find_all('dd')
    
    kobo_info = {}
    
    # タイトルを追加
    kobo_title = soup_li[n].find_all('h3')
    kobo_title = kobo_title[0].text.replace('\u3000', '')
    kobo_info.setdefault('タイトル', kobo_title)
    
    # 応募リンクを追加
    obo_link = ''
    # 川柳の詳細ページのHTML要素の<a>タグのclass='btn-koushiki'の要素を取得
    obo_link = soup_kobo.find_all('a', attrs={'class': 'btn-koushiki'})
    obo_link = obo_link[0].get('href')
    # 応募リンクをkobo_infoに追加
    kobo_info.setdefault('応募リンク', obo_link)
    
    # 公募概要情報の取得・配列に格納
    for j in range(len(soup_kobo_dd)):
        kobo_feature_title = soup_kobo_dt[j].text
        kobo_feature_title = kobo_feature_title.replace('\u3000', ' ')
        
        # 項目名をデータベースのカラム名に変換
        if '締切' == kobo_feature_title:
            kobo_feature_title = '締切日'
            
            # 日付だけを抽出
            kobo_feature = soup_kobo_dd[j].text.replace('\u3000', ' ')
            kobo_feature = kobo_feature.replace('\n', '')
            kobo_feature = kobo_feature.replace('\t', '')
            kobo_feature = kobo_feature.replace('\r', '')
            kobo_feature = re.match('20\d\d.\d\d.\d\d.', kobo_feature)
            kobo_feature =  str(kobo_feature.group())
            kobo_feature = kobo_feature.replace('年', '-')
            kobo_feature = kobo_feature.replace('月', '-')
            kobo_feature = kobo_feature.replace('日', '')
            kobo_info.setdefault(kobo_feature_title, kobo_feature)
            
            continue
            
        elif '賞' == kobo_feature_title:
            kobo_feature_title = '賞金'
        elif '募集内容' == kobo_feature_title:
            kobo_feature_title = '募集内容'
        elif '参加資格' == kobo_feature_title:
            kobo_feature_title = '応募資格'
        elif '主催' == kobo_feature_title:
            kobo_feature_title = '主催'
        else:
            continue
        
        # HTML要素のテキストだけを取り出す
        kobo_feature = soup_kobo_dd[j].text.replace('\u3000', ' ')
        kobo_feature = kobo_feature.replace('\n', '')
        kobo_feature = kobo_feature.replace('\t', '')
        kobo_feature = kobo_feature.replace('\r', '')
        kobo_info.setdefault(kobo_feature_title, kobo_feature)
        
    # kobo_infoの中身を並び替える
    kobo_sort = {}
    kobo_sort.setdefault('タイトル', kobo_info['タイトル'])
    kobo_sort.setdefault('締切日', kobo_info['締切日'])
    kobo_sort.setdefault('賞金', kobo_info['賞金'])
    kobo_sort.setdefault('募集内容', kobo_info['募集内容'])
    kobo_sort.setdefault('応募資格', kobo_info['応募資格'])
    kobo_sort.setdefault('主催', kobo_info['主催'])
    kobo_sort.setdefault('応募リンク', kobo_info['応募リンク'])
    
    # 公募情報を配列に格納
    kobo_info_list.append(kobo_sort)
    
    # サーバへの負荷を下げるためにsleep
    time.sleep(3)
    
    
# csvで出力

df = pd.DataFrame(kobo_info_list)

# excelで文字化けしないようにutf_8_sigでエンコード
df.to_csv('kobo_scraping.csv', encoding='utf_8_sig', index = False)

In [1]:
# 確認用
# kobo_sort

In [3]:
# 確認用
kobo_info_list

[{'タイトル': '作詩・作曲コンテスト ソングコンテストグランプリ 2022作詩部門',
  '締切日': '2022-03-17',
  '賞金': '●グランプリ 山本譲二が歌唱',
  '募集内容': '未発表オリジナルの歌詞※歌唱は山本譲二（テイチクエンタテインメント）',
  '応募資格': '不問',
  '主催': '公益社団法人 日本作曲家協会、一般社団法人 日本作詩家協会',
  '応募リンク': 'https://jacompa.or.jp/songcontest.html'},
 {'タイトル': '第8回 憲法改正（改憲）川柳コンクール',
  '締切日': '2022-03-25',
  '賞金': '●大賞（1名） 賞金3万円、書籍「国民のための憲法改正学の勧め」●佳作（6名） 5000円分の図書カード、書籍「国民のための憲法改正学の勧め」※同一作品が複数の場合は抽選で入選作を決定する',
  '募集内容': '憲法改正を題材にした未発表の川柳（五七五）※改憲反対的内容の作品は受付不可※本人が創作した、他のコンクール等への応募がなされていない作品※過去の応募作品と同一の句は不可',
  '応募資格': '日本国内在住者',
  '主催': '新しい憲法をつくる国民会議（＝自主憲法制定国民会議）',
  '応募リンク': 'http://atarashii-kenpou.jp/event/senryu-contest2022.html'},
 {'タイトル': '第11回 いかなごのくぎ煮文学賞',
  '締切日': '2022-03-31',
  '賞金': '●グランプリ（1点） 賞金2万円、5000円相当の伍魚福珍味セット●準グランプリ（1点） 賞金1万円、5000円相当の伍魚福珍味セット●郵便局賞（1点） 5000円相当の文具類セット、ポスト型貯金箱、5000円相当の伍魚福珍味セット●特選（各部門1点程度） 3000円分のギフトカード、3000円相当の伍魚福珍味セット●いかなごのくぎ煮振興協会賞（数点） 1000円相当の伍魚福珍味セット【ジュニア部門（高校生以下対象）】●グランプリ（1点） 5000円分の図書カード、3000円相当の伍魚福珍味セット●準グランプリ（1点） 3000円分の図書カード、3000円相当の伍魚福珍味セット

In [1]:
import requests
import KOBO_URL
import time
import re
import pandas as pd
from bs4 import BeautifulSoup

# 確認用
df_read = pd.read_csv('kobo_scraping.csv', index_col=0)
# df_read

In [1]:
import requests
import KOBO_URL
import time
import re
from bs4 import BeautifulSoup

# スクレイピング対象のサイトのURLを取得
url1 = KOBO_URL.KOBO_URL1
res = requests.get(url1)

In [2]:
# WebサイトのHTML要素を取得
soup = BeautifulSoup(res.text, 'html.parser')

In [3]:
# soup

In [4]:
# li要素を取得
soup_li = soup.find_all('li', attrs={'class': 'contest-list-item'})

In [5]:
# h3要素を取得
# soup_h3 = soup_li[0].find_all('h3')

In [6]:
# soup_h3

In [7]:
# a要素を取得
# soup_a = soup_li[0].find('a')
# soup_a = soup_a.get('href')

In [8]:
# soup_a

In [9]:
# 応募リンクを配列に格納する
link_list = []
while_cnt = 0
while True:
    try:
        soup_a = soup_li[while_cnt].find('a')
        soup_a = soup_a.get('href')
        link_list.append(soup_a)
        
    except IndexError as e:
        break
        
    while_cnt += 1

In [10]:
# link_list

In [11]:
# soup_a_cnt

In [12]:
kobo_url = link_list[0]

In [13]:
# kobo_url

In [14]:
res_kobo = requests.get(kobo_url)

In [15]:
# res_kobo

In [16]:
# 公募の詳細ページのHTML要素を取得
soup_kobo = BeautifulSoup(res_kobo.text, 'html.parser')

In [17]:
# soup_kobo

In [18]:
soup_kobo_dl = soup_kobo.find_all('dl')

In [19]:
# soup_kobo_dl

In [20]:
# n = 0
# for n in range(30):
#     soup_kobo_dt = soup_kobo_dl[n].find_all('dt')
        
#     if '締切' == soup_kobo_dt[0].text:
#         break
        
# soup_kobo_dd = soup_kobo_dl[n].find_all('dd')
# soup_kobo_dd

In [21]:
while_cnt = 0
while True:
    try:
        soup_kobo_dt = soup_kobo_dl[while_cnt].find_all('dt')
        
        if '締切' == soup_kobo_dt[0].text:
            break
        
    except IndexError as e:
        break
        
    while_cnt += 1

In [22]:
soup_kobo_dt = soup_kobo_dl[while_cnt].find_all('dt')
soup_kobo_dd = soup_kobo_dl[while_cnt].find_all('dd')

In [23]:
# soup_kobo_dt[0].text

In [24]:
# soup_kobo_dd

In [25]:
# kobo_title = soup_kobo_dt[0].text
# kobo_feature = soup_kobo_dd[0].text.replace('\n', '')
# kobo_feature = kobo_feature.replace('\t', '')

kobo_title_list = []
kobo_feature_list = []

In [26]:
# len(soup_kobo_dd)

In [27]:
for n in range(len(soup_kobo_dd)):
    kobo_title = soup_kobo_dt[n].text
    kobo_feature = soup_kobo_dd[n].text.replace('\n', '')
    kobo_feature = kobo_feature.replace('\t', '')
    kobo_title_list.append(kobo_title)
    kobo_feature_list.append(kobo_feature)
    

In [28]:
# kobo_title_list

In [29]:
# kobo_feature_list

In [30]:
# kobo_info = {}

# for n in range(len(soup_kobo_dd)):
#     kobo_feature_title = soup_kobo_dt[n].text
#     kobo_feature = soup_kobo_dd[n].text.replace('\n', '')
#     kobo_feature = kobo_feature.replace('\t', '')
#     kobo_feature = kobo_feature.replace('\u3000', ' ')
#     # kobo_info.setdefault(soup_kobo_dt[n].text, kobo_feature)
#     kobo_info.setdefault(kobo_feature_title, kobo_feature)

In [31]:
# soup_kobo_dt

In [32]:
# soup_kobo_dt[0].text

In [33]:
# soup_kobo_dd[1].text

In [34]:
# kobo_info_dict = {}
kobo_info_list = []

# for n in range(len(link_list)):
for n in range(2):
    # 情報を取得する公募ページに移動してHTTPを取得
    kobo_url = link_list[n]
    res_kobo = requests.get(kobo_url)
    soup_kobo = BeautifulSoup(res_kobo.text, 'html.parser')
    soup_kobo_dl = soup_kobo.find_all('dl')
    # soup_kobo_dt = soup_kobo_dl[3].find_all('dt')
    # soup_kobo_dd = soup_kobo_dl[3].find_all('dd')
    
    while_cnt = 0
    while True:
        try:
            soup_kobo_dt = soup_kobo_dl[while_cnt].find_all('dt')
            if '締切' == soup_kobo_dt[0].text:
                break
        except IndexError as e:
            break
        while_cnt += 1
    
    soup_kobo_dd = soup_kobo_dl[while_cnt].find_all('dd')
    
    kobo_info = {}
    
    # タイトルを追加
    kobo_title = soup_li[n].find_all('h3')
    kobo_title = kobo_title[0].text.replace('\u3000', '')
    kobo_info.setdefault('タイトル', kobo_title)
    
    # 応募リンクを追加
    obo_link = ''
    # 川柳の詳細ページのHTML要素の<a>タグのclass='btn-koushiki'の要素を取得
    obo_link = soup_kobo.find_all('a', attrs={'class': 'btn-koushiki'})
    obo_link = obo_link[0].get('href')
    # 応募リンクをkobo_infoに追加
    kobo_info.setdefault('応募リンク', obo_link)
    
    # 公募概要情報の取得・配列に格納
    for j in range(len(soup_kobo_dd)):
        kobo_feature_title = soup_kobo_dt[j].text
        kobo_feature_title = kobo_feature_title.replace('\u3000', ' ')
        
        # 項目名をデータベースのカラム名に変換
        if '締切' == kobo_feature_title:
            kobo_feature_title = '締切日'
            
            # 日付だけを抽出
            kobo_feature = soup_kobo_dd[j].text.replace('\u3000', ' ')
            kobo_feature = kobo_feature.replace('\n', '')
            kobo_feature = kobo_feature.replace('\t', '')
            kobo_feature = kobo_feature.replace('\r', '')
            kobo_feature = re.match('20\d\d.\d\d.\d\d.', kobo_feature)
            kobo_feature =  str(kobo_feature.group())
            kobo_feature = kobo_feature.replace('年', '-')
            kobo_feature = kobo_feature.replace('月', '-')
            kobo_feature = kobo_feature.replace('日', '')
            kobo_info.setdefault(kobo_feature_title, kobo_feature)
            
            continue
            
        elif '賞' == kobo_feature_title:
            kobo_feature_title = '賞金'
        elif '募集内容' == kobo_feature_title:
            kobo_feature_title = '募集内容'
        elif '参加資格' == kobo_feature_title:
            kobo_feature_title = '応募資格'
        elif '主催' == kobo_feature_title:
            kobo_feature_title = '主催'
        else:
            continue
        
        # HTML要素のテキストだけを取り出す
        kobo_feature = soup_kobo_dd[j].text.replace('\u3000', ' ')
        kobo_feature = kobo_feature.replace('\n', '')
        kobo_feature = kobo_feature.replace('\t', '')
        kobo_feature = kobo_feature.replace('\r', '')
        kobo_info.setdefault(kobo_feature_title, kobo_feature)
        
    # kobo_infoの中身を並び替える
    kobo_sort = {}
    kobo_sort.setdefault('タイトル', kobo_info['タイトル'])
    kobo_sort.setdefault('締切日', kobo_info['締切日'])
    kobo_sort.setdefault('賞金', kobo_info['賞金'])
    kobo_sort.setdefault('募集内容', kobo_info['募集内容'])
    kobo_sort.setdefault('応募資格', kobo_info['応募資格'])
    kobo_sort.setdefault('主催', kobo_info['主催'])
    kobo_sort.setdefault('応募リンク', kobo_info['応募リンク'])
    
    # 公募情報を配列に格納
    # kobo_title = soup_li[n].find_all('h3')
    # kobo_title = kobo_title[0].text.replace('\u3000', '')
    # kobo_info_dict.setdefault(kobo_title, kobo_info)
    
    kobo_info_list.append(kobo_sort)
    
    # サーバへの負荷を下げるためにsleep
    time.sleep(3)

In [35]:
# 確認用
kobo_sort

{'タイトル': '第3回 私のSDGsコンテスト',
 '締切日': '2022-02-13',
 '賞金': '●大賞 10万円分の図書カード●シャボン玉石けん賞 賞品●北九州市長賞 賞品●朝日新聞社賞 折り畳み自転車●魚町銀天街振興組合賞 商品券●スターフライヤー賞 賞品●ギラヴァンツ北九州賞 選手サイン入りユニフォーム●資さんうどん賞 資さんうどん商品セット●小学生優秀賞 1万円分の図書カード●中学生優秀賞 1万円分の図書カード●高校生優秀賞 1万円分の図書カード※川柳部門・フォト部門それぞれに大賞・各賞あり',
 '募集内容': 'テーマにそった川柳または写真※未発表のオリジナル作品に限る【テーマ】17あるSDGsの目標（公式ホームページを参照）から一つを選び、自分が身近にできること、思うことを考えること【部門】（1）川柳部門（2）フォト部門',
 '応募資格': '不問',
 '主催': 'シャボン玉石けん株式会社、株式会社朝日新聞社、北九州市（SDGs未来都市）',
 '応募リンク': 'https://www.asahi.com/ads/my-sdgs-contest/'}

In [36]:
# 確認用
kobo_info_list

[{'タイトル': '第6回 オリックス 働くパパママ川柳 作品募集',
  '締切日': '2022-02-11',
  '賞金': '●大賞（1句） 賞金20万円、オリックスグループが運営する4施設から選べる1泊2食付きホテル宿泊券（1組2名分）●パパママ目線賞（6句） ダイソン Dyson Supersonic Ionic ヘアドライヤー●子ども目線賞（1句） ブルーノ ホットプレート グランデサイズ●じぃじばぁば目線賞（1句） 2万円分のダスキン 選んで使えるおそうじギフトカード●優秀賞（10句） 1万円分のAmazonギフト券',
  '募集内容': '働きながら子育てに奮闘するパパとママの日常をテーマとする川柳※応募者の自作未発表作品に限る',
  '応募資格': '既婚未婚、年齢不問',
  '主催': 'オリックスグループ',
  '応募リンク': 'https://www.orix.co.jp/grp/move_on/entry/orix-senryu/vol6'},
 {'タイトル': '第3回 私のSDGsコンテスト',
  '締切日': '2022-02-13',
  '賞金': '●大賞 10万円分の図書カード●シャボン玉石けん賞 賞品●北九州市長賞 賞品●朝日新聞社賞 折り畳み自転車●魚町銀天街振興組合賞 商品券●スターフライヤー賞 賞品●ギラヴァンツ北九州賞 選手サイン入りユニフォーム●資さんうどん賞 資さんうどん商品セット●小学生優秀賞 1万円分の図書カード●中学生優秀賞 1万円分の図書カード●高校生優秀賞 1万円分の図書カード※川柳部門・フォト部門それぞれに大賞・各賞あり',
  '募集内容': 'テーマにそった川柳または写真※未発表のオリジナル作品に限る【テーマ】17あるSDGsの目標（公式ホームページを参照）から一つを選び、自分が身近にできること、思うことを考えること【部門】（1）川柳部門（2）フォト部門',
  '応募資格': '不問',
  '主催': 'シャボン玉石けん株式会社、株式会社朝日新聞社、北九州市（SDGs未来都市）',
  '応募リンク': 'https://www.asahi.com/ads/my-sdgs-contest/'}]

In [37]:
# csvで出力
import pandas as pd

df = pd.DataFrame(kobo_info_list)

# excelで文字化けしないようにutf_8_sigでエンコード
df.to_csv('kobo_scraping.csv', encoding='utf_8_sig', index = False)

In [36]:
# kobo_info_list

In [38]:
df_read = pd.read_csv('kobo_scraping.csv', index_col=0)
df_read

,締切日,賞金,募集内容,応募資格,主催,応募リンク
タイトル,,,,,,
第6回 オリックス 働くパパママ川柳 作品募集,2022-02-11,●大賞（1句） 賞金20万円、オリックスグループが運営する4施設から選べる1泊2食付きホテル...,働きながら子育てに奮闘するパパとママの日常をテーマとする川柳※応募者の自作未発表作品に限る,既婚未婚、年齢不問,オリックスグループ,https://www.orix.co.jp/grp/move_on/entry/orix-...
第3回 私のSDGsコンテスト,2022-02-13,●大賞 10万円分の図書カード●シャボン玉石けん賞 賞品●北九州市長賞 賞品●朝日新聞社賞 ...,テーマにそった川柳または写真※未発表のオリジナル作品に限る【テーマ】17あるSDGsの目標（...,不問,シャボン玉石けん株式会社、株式会社朝日新聞社、北九州市（SDGs未来都市）,https://www.asahi.com/ads/my-sdgs-contest/


In [84]:
# 締切日をchar型からdate型に変換する

str_date = '2022年02月11日 (金)作品提出・応募締切、必着'
str_date

'2022年02月11日 (金)作品提出・応募締切、必着'

In [85]:
import re

# pattern = '(20\d\d.月)'
pattern = '20\d\d.\d\d.\d\d.'

str_date = re.match(pattern, str_date)
str_date =  str(str_date.group())
str_date

'2022年02月11日'

In [86]:
str_date = str_date.replace('年', '-')
str_date = str_date.replace('月', '-')
str_date = str_date.replace('日', '')
str_date

'2022-02-11'

In [78]:
from datetime import datetime

mydate = datetime.strptime(str_date, '%Y年%m月%d日')
print(mydate)

2022-02-11 00:00:00
